In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from config_variables import var_credentials_location, var_gcs_connector
import requests
import pandas as pd
from io import StringIO


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Configuration for Spark Cluster
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('GCS Ingestion') \
    .set("spark.jars", var_gcs_connector) \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", var_credentials_location)

In [3]:
#  Configuring Hadoop

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", var_credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

23/08/09 09:47:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Initialising Spark Session
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate() 

In [9]:
# Reading GCS Data into Spark DF
df_15 = spark.read \
    .option("header", "true") \
    .csv("gs://pipelineproject01-data-bronze-bucket/raw/2015/*")

In [10]:
df_15.printSchema()

root
 |-- Year: string (nullable = true)
 |-- UnqID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Weather: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Round: string (nullable = true)
 |-- Dir: string (nullable = true)
 |-- Path: string (nullable = true)
 |-- Mode: string (nullable = true)
 |-- Count: string (nullable = true)



In [7]:
df_15.show()

+----+--------+----------+-------+--------+-------+-----+----------+----+--------------+-----+
|Year|   UnqID|      Date|Weather|    Time|    Day|Round|       Dir|Path|          Mode|Count|
+----+--------+----------+-------+--------+-------+-----+----------+----+--------------+-----+
|2015|CSHCY005|10/10/2014|    Wet|06:00:00|Weekday|    A|Northbound|null|Private cycles|    0|
|2015|CSHCY005|10/10/2014|    Wet|06:15:00|Weekday|    A|Northbound|null|Private cycles|    0|
|2015|CSHCY005|10/10/2014|    Wet|06:30:00|Weekday|    A|Northbound|null|Private cycles|    0|
|2015|CSHCY005|10/10/2014|    Dry|06:45:00|Weekday|    A|Northbound|null|Private cycles|    0|
|2015|CSHCY005|10/10/2014|    Dry|07:00:00|Weekday|    A|Northbound|null|Private cycles|    0|
|2015|CSHCY005|10/10/2014|    Dry|07:15:00|Weekday|    A|Northbound|null|Private cycles|    2|
|2015|CSHCY005|10/10/2014|    Dry|07:30:00|Weekday|    A|Northbound|null|Private cycles|    0|
|2015|CSHCY005|10/10/2014|    Dry|07:45:00|Weekday

### DataSet Info

UnqID - UUID for London locations

Count - Disaggregated by site, direction, 15-min period and by path and mode classifications that vary depending on the area and year

Mode - Drop this column

Path - Drop this column

In [7]:
spark

In [8]:
# spark.stop()